# soynlp로 자연어 처리
* https://github.com/lovit/soynlp

In [1]:
# 출력이 너무 길어지지 않게하기 위해 찍지 않도록 했으나 
# 실제 학습 할 때는 아래 두 줄을 주석처리 하는 것을 권장한다.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

0.21.0
1.14.0


In [3]:
petitions = pd.read_csv('data/petition_sampled.csv')
# 데이터의 크기가 어느정도인지 본다.
petitions.shape

(8029, 8)

In [4]:
petitions.head()

,article_id,start,end,answered,votes,category,title,content
0,58,2017-08-19,2017-11-17,0,21,일자리,국토교통부와 한국주택협회가 행한 부당한 행위와 권력남용에 대한 내용을 청원드립니다.,안녕하세요? 존경하고 지지하는 문재인 대통령님!\n저는 성남시 분당구 정자동 주택전...
1,63,2017-08-20,2017-09-04,0,1,보건복지,살려주세요..,안녕하십니까?\n저는 올해 63세된 홀로 사는 늙은 여자입니다...\n작년 중복날 ...
2,136,2017-08-20,2017-11-18,0,4,육아/교육,고등학교 교육 내용 수준을 낮춰주시고 실용적인 내용을 담아주세요!,저는 광주에 사는 중3 학생입니다. 고등학교 가기 직전의 학년이라 어느 때보다 고등...
3,141,2017-08-20,2017-08-27,0,0,기타,한국문화에 창조적요소를 심자,안녕하십니까\n저는 92년 한국을 알게된 종국동포 입니다.\n[저는 한 중소기업에...
4,148,2017-08-20,2017-11-18,0,7,외교/통일/국방,다문화정책 및 할랄 인증 제도,대한민국과 국민을 위해 밤낮 없이 수고하시는 대통령을 비롯한 위정자 분들께\n대한민...


In [5]:
petitions_content = ' '.join(str(petitions['content']))

In [6]:
class Sentences:
    def __init__(self, fname):
        self.fname = fname
        self.length = 0
    def __iter__(self):
        for doc in self.fname:
            doc = doc.strip()
            if not doc:
                continue
            for sent in doc.split(' '):
                yield sent
    def __len__(self):
        if self.length == 0:
            for doc in self.fname:
                doc = doc.strip()
                if not doc:
                    continue
                self.length += len(doc.split(' '))
        return self.length

In [7]:
corpus_fname = petitions_content
sentences = Sentences(corpus_fname)
print('num sentences = %d' % len(sentences))

num sentences = 2633


In [8]:
corpus_fname = petitions['title']
sentences = Sentences(corpus_fname)
print('num sentences = %d' % len(sentences))

num sentences = 37956


In [9]:
%%time
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_count=100,
                               min_cohesion_forward=0.05, 
                               min_right_branching_entropy=0.0)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.114 Gb
all cohesion probabilities was computed. # words = 101
all branching entropies was computed # words = 91
all accessor variety was computed # words = 91
CPU times: user 544 ms, sys: 62 ms, total: 606 ms
Wall time: 721 ms


In [10]:
import itertools
dict(itertools.islice(words.items(), 5))

{'신': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=189, rightside_frequency=0),
 '와': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=0, rightside_frequency=164),
 '용': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=0, rightside_frequency=120),
 '평': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=126, rightside_frequency=0),
 '학': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency

In [11]:
len(words)

198

In [12]:
words['국민']

Scores(cohesion_forward=0.4335347432024169, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=287, rightside_frequency=0)

words는 {word:Score} 형식의 dictionary입니다. Score는 soynlp/word.py에 구현되어있는 namedtuple입니다.

In [13]:
print('type: %s\n' % type(words['가상화폐']))
print(words['가상화폐'])

type: <class 'soynlp.word._word.Scores'>

Scores(cohesion_forward=0.7862404931722864, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=261, rightside_frequency=0)


In [14]:
print('type: %s\n' % type(words['청원']))
print(words['청원'])

type: <class 'soynlp.word._word.Scores'>

Scores(cohesion_forward=0.5051440329218106, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=-0.0, left_accessor_variety=0, right_accessor_variety=1, leftside_frequency=491, rightside_frequency=0)


In [15]:
def word_score(score):
    import math
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (word, 
                                   score.leftside_frequency, 
                                   score.cohesion_forward,
                                   score.right_branching_entropy
                                  ))

단어   (빈도수, cohesion, branching entropy)

청소년     (346, 0.597, 0.692)
폐지     (570, 0.857, -0.000)
합니다     (195, 0.823, -0.000)
출국금지     (321, 0.809, 0.000)
처벌     (251, 0.789, 0.000)
바랍니다     (102, 0.787, 0.000)
가상화폐     (261, 0.786, 0.000)
반대합니다     (166, 0.780, 0.000)
반대     (344, 0.766, -0.000)
만들어     (102, 0.749, 0.000)
부탁드립니다     (101, 0.747, 0.000)
이명박     (537, 0.731, 0.000)
합니     (210, 0.729, -0.000)
가상화     (282, 0.725, -0.000)
합니다.     (106, 0.717, 0.000)
해주세요     (215, 0.715, 0.000)
소년법     (179, 0.712, 0.000)
청원합니다     (238, 0.703, -0.000)
문재인     (139, 0.685, 0.000)
금지     (160, 0.675, 0.000)
청원합니다.     (130, 0.669, 0.000)
주세요     (257, 0.665, 0.000)
출국     (400, 0.660, -0.000)
만들     (119, 0.654, -0.000)
청소년보호법     (112, 0.649, 0.000)
조두순     (186, 0.624, 0.000)
요청     (137, 0.614, 0.000)
국회의원     (100, 0.533, 0.000)
청원     (491, 0.505, -0.000)
부탁드     (108, 0.498, -0.000)


In [16]:
score

Scores(cohesion_forward=0.4982728791224398, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=-0.0, left_accessor_variety=0, right_accessor_variety=1, leftside_frequency=108, rightside_frequency=0)

In [17]:
from soynlp.tokenizer import RegexTokenizer, LTokenizer, MaxScoreTokenizer

tokenizer = RegexTokenizer()
tokenizer

In [18]:
sample_title = petitions['title'][2]
sample_title

'고등학교 교육 내용 수준을 낮춰주시고 실용적인 내용을 담아주세요!'

In [19]:
sample_content = petitions['content'][2]
sample_content

"저는 광주에 사는 중3 학생입니다. 고등학교 가기 직전의 학년이라 어느 때보다 고등학교 선행학습을 학원다니며 치열하게 하고 있고 주위로부터 고등학교에 가면 국어가 어렵다느니 수학이 어렵다느니 과학이 어렵다느니 이런 말을 많이 듣고 있습니다. 실제로 제가 고등학교 선행학습을 하면 국어, 수학, 영어가 중학교에서 배우던 내용과 다르게 수준이 갑자기 올라가서 ' 아 정말 고등학교를 선행 안 하고 가면 성적이 안나오고 힘들다는 말이 그 말이구나'하는 생각이 듭니다. 고등학교 교육 내용 수준을 낮춰주세요. 그리고 솔직히 말해서 고등학교에서 배우는 내용들은 정말 생활 속에서 잘 쓰이지 않는 것 같아 배울 때 체감도도 떨어지는 것 같습니다. 예를 들어 수학의 미적분 내용은 수학을 전문적으로 다루는 사람이 아니면 실생활에서 사용되지 않는 것 같습니다. 저는 실생활에서 정말 잘 쓰이고 꼭 알아야 될 것들을 배워야 한다고 생각합니다. 고등학교를 위해 제 주변에서도 수 많은 친구들이 선행학습 위해 학원을 다니느라 스트레스도 많이 받습니다. 고등학교 교육 내용을 바꾼다면 사교육도 줄어들지 않을까요"

### RegexTokenizer

In [20]:
tokened_title = tokenizer.tokenize(sample_title)
tokened_title

['고등학교', '교육', '내용', '수준을', '낮춰주시고', '실용적인', '내용을', '담아주세요', '!']

In [21]:
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:10]

['저는', '광주에', '사는', '중', '3', '학생입니다', '.', '고등학교', '가기', '직전의']

In [22]:
print(len(tokened_title))
print(len(tokened_content))

9
148


In [23]:
import re
from bs4 import BeautifulSoup
def preprocessing(text):
    # HTML 제거
    text = BeautifulSoup(text, 'html.parser').get_text()
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    return text

In [24]:
%time sentences = petitions['content'].apply(preprocessing)

CPU times: user 719 ms, sys: 89.7 ms, total: 808 ms
Wall time: 854 ms


In [25]:
%time sentences = sentences.apply(tokenizer.tokenize)
sentences[:3]

CPU times: user 7.27 s, sys: 117 ms, total: 7.39 s
Wall time: 7.9 s


0    [안녕하세요, ?, 존경하고, 지지하는, 문재인, 대통령님, !, 저는, 성남시, ...
1    [안녕하십니까, ?, 저는, 올해, 63, 세된, 홀로, 사는, 늙은, 여자입니다,...
2    [저는, 광주에, 사는, 중, 3, 학생입니다, ., 고등학교, 가기, 직전의, 학...
Name: content, dtype: object

In [26]:
sentences[2][:10]

['저는', '광주에', '사는', '중', '3', '학생입니다', '.', '고등학교', '가기', '직전의']

In [27]:
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [28]:
# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(sentences, min_count=1)

model

2018-05-20 09:27:07,183 : INFO : 'pattern' package not found; tag filters are not available for English
2018-05-20 09:27:07,188 : INFO : collecting all words and their counts
2018-05-20 09:27:07,190 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-20 09:27:07,645 : INFO : collected 206569 word types from a corpus of 998862 raw words and 8029 sentences
2018-05-20 09:27:07,646 : INFO : Loading a fresh vocabulary
2018-05-20 09:27:08,654 : INFO : min_count=1 retains 206569 unique words (100% of original 206569, drops 0)
2018-05-20 09:27:08,657 : INFO : min_count=1 leaves 998862 word corpus (100% of original 998862, drops 0)
2018-05-20 09:27:09,269 : INFO : deleting the raw counts dictionary of 206569 items
2018-05-20 09:27:09,274 : INFO : sample=0.001 downsamples 14 most-common words
2018-05-20 09:27:09,275 : INFO : downsampling leaves estimated 928590 word corpus (93.0% of prior 998862)
2018-05-20 09:27:09,277 : INFO : estimated required memory for 206569

In [29]:
model_name = '1minwords'
model.save(model_name)

2018-05-20 09:27:19,314 : INFO : saving Word2Vec object under 1minwords, separately None
2018-05-20 09:27:19,316 : INFO : storing np array 'syn0' to 1minwords.wv.syn0.npy
2018-05-20 09:27:19,440 : INFO : not storing attribute syn0norm
2018-05-20 09:27:19,442 : INFO : storing np array 'syn1neg' to 1minwords.syn1neg.npy
2018-05-20 09:27:19,557 : INFO : not storing attribute cum_table
2018-05-20 09:27:20,150 : INFO : saved 1minwords


In [30]:
model.wv['청원']

array([-1.0440398 , -1.0443611 , -0.5516773 ,  0.6102202 ,  1.8554325 ,
        0.06375427,  1.6854692 ,  0.5142126 , -1.3391005 , -0.32294902,
       -0.68601817,  1.5004176 , -0.24300487,  0.18031558, -0.51142156,
       -0.08511166,  0.7174916 , -0.3621651 ,  1.3833635 ,  0.55446476,
       -0.9949153 , -0.7331671 ,  0.59646815, -0.7953661 ,  0.0874754 ,
        1.5770007 ,  0.88728523, -1.6485964 ,  0.4021291 , -0.49264896,
       -1.1193633 ,  0.6225354 , -0.31333858,  0.19409972,  0.1427176 ,
        1.3675289 ,  1.2637956 , -0.07709076, -0.11442575,  0.1845911 ,
        1.0062613 ,  0.8003481 ,  0.13619499, -1.2933979 , -0.2974665 ,
       -0.9859013 ,  0.5246085 ,  1.1209651 , -0.56296647,  0.44686767,
       -0.8310985 ,  0.21361355, -0.2939626 , -0.3863641 ,  0.40865374,
       -0.8398327 , -0.37009218,  0.5751925 ,  0.8027985 , -1.1924516 ,
       -0.81842595, -0.16124794, -0.1353309 ,  0.09152485, -1.2814677 ,
       -0.47796848, -0.63738215, -0.5891517 , -1.1685637 , -0.13

In [31]:
# 유사도가 없는 단어 추출
model.wv.doesnt_match('국민 청원 답변 전안법'.split())

2018-05-20 09:27:20,168 : INFO : precomputing L2-norms of word weight vectors


'답변'

In [32]:
# 유사도가 없는 단어 추출
model.wv.doesnt_match('이명박 박근혜 대통령 김정은'.split())

'김정은'

In [33]:
# 유사도가 없는 단어 추출
model.wv.doesnt_match('전안법 전기 안전 의류 교통'.split())

'전안법'

In [34]:
# 가장 유사한 단어를 추출
model.wv.most_similar('전안법')

[('하죠', 0.990835964679718),
 ('말로만', 0.9899716973304749),
 ('뉴스보면', 0.9896503686904907),
 ('그냥', 0.989278256893158),
 ('뭐합니까', 0.9889724850654602),
 ('묻고싶습니다', 0.9882982969284058),
 ('이거', 0.9879079461097717),
 ('청소년이라고', 0.9877943396568298),
 ('안됩니다', 0.9875889420509338),
 ('그러니까', 0.9872233271598816)]

In [35]:
# 가장 유사한 단어를 추출
model.wv.most_similar('대통령')

[('박근혜', 0.9973636865615845),
 ('평소', 0.995707094669342),
 ('대통령의', 0.9951146841049194),
 ('노무현', 0.994614839553833),
 ('정부', 0.9944683313369751),
 ('개인적인', 0.9943782687187195),
 ('청와대', 0.9940354228019714),
 ('군에', 0.9939010739326477),
 ('언론이나', 0.9935199618339539),
 ('법률을', 0.993385910987854)]

In [36]:
# 가장 유사한 단어를 추출
model.wv.most_similar('가상화폐')

[('명백히', 0.99812912940979),
 ('사건이', 0.9979899525642395),
 ('폐쇄', 0.9979604482650757),
 ('블록체인', 0.9978711009025574),
 ('기존의', 0.9978052973747253),
 ('보면', 0.9977903366088867),
 ('취지는', 0.997608482837677),
 ('학교는', 0.9975537061691284),
 ('내용으로', 0.9974758625030518),
 ('사건을', 0.9973389506340027)]